In [ ]:
import os

try:
    if CONFIGURED_DIRS:  # type: ignore
        pass
except:
    PLAYGROUND_DIR = os.getcwd()
    os.chdir("../src")

CONFIGURED_DIRS = True

print(f"Playground directory: {PLAYGROUND_DIR}")  # type: ignore
print(f"Working directory: {os.getcwd()}")

In [ ]:
import sys
import importlib
from IPython.display import Audio

from generation.instruments.voice import Voice
from generation.instruments.piano import Piano
from generation.instruments.bass_drum import BassDrum
from generation.instruments.snare_drum import SnareDrum

from export.arrangement import Arrangement, ArrangementExportConfig
from export.arrangement_generator import ArrangementGenerator

In [ ]:
sys.path.append("../")  # allow modules one directory above to be accessible

In [ ]:
def create_song(name: str) -> Arrangement:
    song = importlib.import_module(f"playground.songs.{name}")
    arrangement_generator: ArrangementGenerator = song.get_song()

    arrangement_generator.add_instrument("Voice", Voice)
    arrangement_generator.add_instrument("Stride Piano", Piano)
    arrangement_generator.add_instrument("Bass Drum", BassDrum)
    arrangement_generator.add_instrument("Snare Drum", SnareDrum)

    return arrangement_generator.generate()


def export_song(name: str) -> str:
    output_path = os.path.join(PLAYGROUND_DIR, "output", f"{name}.wav")
    create_song(name).export(ArrangementExportConfig(output_path=output_path))
    return output_path

In [ ]:
Audio(export_song("first"))

In [ ]:
Audio(export_song("happy_birthday"))